# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор (любой) с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [70]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier as T
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.naive_bayes import MultinomialNB as bayes
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sklearn.datasets import make_classification

from razdel import tokenize as rt

In [71]:
data = pd.read_csv('labeled.csv')

In [72]:
def razdel_tokenizer(text):
    return [token.text for token in list(rt(text))]

In [73]:
train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [74]:
vectorizer1 = TfidfVectorizer (tokenizer = razdel_tokenizer)
X1 = vectorizer1.fit_transform(train.comment)
X1_test = vectorizer1.transform(test.comment)
vectorizer1.get_feature_names_out()

c:\Users\Alex\scoop\apps\python\current\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


array(['!', '!!', '!!!', ..., 'ёмкостью', 'ёпта', 'ёта'], dtype=object)

In [75]:
vectorizer2 = TfidfVectorizer () #tokenizer = default
X2 = vectorizer2.fit_transform(train.comment)
X2_test = vectorizer2.transform(test.comment)
vectorizer2.get_feature_names_out()

array(['00', '000', '0015', ..., 'ёмкостью', 'ёпта', 'ёта'], dtype=object)

In [76]:
y1 = train.toxic.values
y1_test = test.toxic.values
y2 = train.toxic.values
y2_test = test.toxic.values

In [77]:
clf = RFC(min_samples_leaf = 2, class_weight = 'balanced')
clf.fit(X1, y1)
preds1 = clf.predict(X1_test)
print(classification_report(y1_test, preds1, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.86      0.83      0.85       969
         1.0       0.68      0.73      0.71       473

    accuracy                           0.80      1442
   macro avg       0.77      0.78      0.78      1442
weighted avg       0.80      0.80      0.80      1442



In [78]:
clf = RFC(min_samples_leaf = 2, class_weight = 'balanced')
clf.fit(X2, y2)
preds2 = clf.predict(X2_test)
print(classification_report(y2_test, preds2, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.85      0.85      0.85       969
         1.0       0.69      0.70      0.69       473

    accuracy                           0.80      1442
   macro avg       0.77      0.77      0.77      1442
weighted avg       0.80      0.80      0.80      1442



Встроенный токенизатор побеждает

## Задание 2 (3 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к моделям:   
а) один классификатор должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров (можно ставить разные параметры tfidfvectorizer и countvectorizer)  
в) у классификатора должно быть задано вручную как минимум 2 параметра (по возможности)  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

*random_seed не считается за параметр

In [79]:
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

In [80]:
vectorizerL = TfidfVectorizer (min_df=4, max_df=0.1, max_features=10000, stop_words=russian_stopwords, ngram_range=(1, 3))
X_for_L = vectorizerL.fit_transform(train.comment)
X_for_L_test = vectorizerL.transform(test.comment)
#vectorizerTF.get_feature_names_out()

In [81]:
vectorizerB = CountVectorizer (min_df=4, max_df=0.3, max_features=10000, stop_words=russian_stopwords, ngram_range=(1, 3))
X_for_B = vectorizerB.fit_transform(train.comment)
X_for_B_test = vectorizerB.transform(test.comment)
#vectorizerC.get_feature_names_out()

In [82]:
yL = train.toxic.values
yL_test = test.toxic.values
yB = train.toxic.values
yB_test = test.toxic.values

In [83]:
clfL = LogisticRegression(penalty='l2', class_weight= 'balanced')
clfL.fit(X_for_L, yL)
predsL = clfL.predict(X_for_L_test)
print(classification_report(yL_test, predsL, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.91      0.88      0.89       969
         1.0       0.76      0.81      0.79       473

    accuracy                           0.86      1442
   macro avg       0.84      0.85      0.84      1442
weighted avg       0.86      0.86      0.86      1442



In [84]:
clfB = bayes(alpha=1, force_alpha=True)
clfB.fit(X_for_B, yB)
predsB = clfB.predict(X_for_B_test)
print(classification_report(yB_test, predsB, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.87      0.92      0.90       969
         1.0       0.82      0.71      0.76       473

    accuracy                           0.86      1442
   macro avg       0.85      0.82      0.83      1442
weighted avg       0.85      0.86      0.85      1442



In [85]:
predsB_proba = clfB.predict_proba(X_for_B_test)
predsL_proba = clfL.predict_proba(X_for_L_test)

In [86]:
from collections import Counter
from pprint import pprint

In [87]:
counterB = Counter({comm: prob[1] for comm, prob in zip(data.comment, predsB_proba)})
B_fin = counterB.most_common(10)
pprint(B_fin)

[('А вы писать по-докторски умеете?\n', np.float64(1.0)),
 ('Истинно так. Но мрии жидов вертят шариком несколько тысяч лет. Мрии хохлов '
  'же больше напоминают бредни душевнобольных. Тут я более доверяю иудеям. Они '
  'вызывают больше доверия, более ответственны и потенциальны. Мой выбор - '
  'жиды.\n',
  np.float64(1.0)),
 ('а почему с кубани начали, а не с москвы?', np.float64(1.0)),
 ('Очень даже разделимы. Достаточно знать мотивы гомосексуального поведения у '
  'животных (которых Вы, как мне кажется, не знаете), и всё становится на '
  'места. А, как Вы выразились, перманентное поведение (т.е. однозначное '
  'предпочтение партнёров своего пола и игнорирование партнёров '
  'противоположного пола) наблюдалось только у баранов. Пути эволюции, '
  'конечно, неисповедимы, но я сомневаюсь, что у геев были бараны среди '
  'предков.',
  np.float64(1.0)),
 ('о ужас, не стерпела мужитская психика шума детского и вышел герой в бой '
  'неравный, супротив детей и женщин и прогнал он ор

In [88]:
counterL = Counter({comm: prob[1] for comm, prob in zip(data.comment, predsL_proba)})
L_fin = counterL.most_common(10)
pprint(L_fin)

[('Действительно, с русским у тебя проблемы. Я тебе уже сказал - свободен, что '
  'бля непонятно?Полностью соответствуешь своему нику, осёл. В игнор, '
  'нерусь.\n',
  np.float64(0.9788012335263476)),
 ('Не дает на 4м свидании-не стоит тратить время.',
  np.float64(0.9777824457799706)),
 ('А почему полицаи 23 февраля отмечают? Они ж за немцев были?',
  np.float64(0.9773828168071238)),
 ('Еще раз- ПРОШУ, прочитать мой комментарий. Внимательно.\n',
  np.float64(0.9773580057214164)),
 ('Шизофазия (речевая разорванность) симптом психических расстройств, '
  'выражающийся в нарушении структуры речи, при которой, в отличие от речевой '
  'бессвязности (потока несвязанных слов), фразы строятся правильно 1 , однако '
  'не несут никакой смысловой нагрузки, а содержание речи соответствует '
  'содержанию бреда 2 . Это расстройство речи отражает разорванность мышления '
  'пациентов 3 . Это расстройство характерно прежде всего для шизофрении. '
  'Понятие шизофазии было введено Э. Крепелином в

In [89]:
same = list()
for i in L_fin:
    for j in B_fin:
        if i==j and j not in same:
            same.append(j)
same

[]

одинаковых текстов нет, тексты действительно токсичные

## Задание 3 (4 балла - 1 балл за каждый классификатор)

Для классификаторов Logistic Regression, Decision Trees, Naive Bayes, RandomForest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию. 
Также как и в предыдущем задании у классификаторов должно быть задано вручную как минимум 2 параметра (по возможности, f1 мера каждого из классификаторов должна быть минимум 0.75

In [90]:
counter_tB = Counter({tok: feature
        for tok, feature
        in zip(
            vectorizerB.vocabulary_,
            clfB.feature_log_prob_[1]
        )}
)
counter_tB.most_common(5)

[('всё всё', np.float64(-3.916006829002055)),
 ('десять лет', np.float64(-5.058790335983302)),
 ('учли', np.float64(-5.379262231258074)),
 ('личного', np.float64(-5.739116828114186)),
 ('устройства', np.float64(-5.745506626212957))]

In [ ]:
counter_tL = Counter({tok: feature
        for tok, feature
        in zip(
            vectorizerB.vocabulary_,
            #здесь не знаю, что использовать, чтобы получить значения
        )}
)
counter_tL.most_common(5)

In [91]:
vectorizerRFC = TfidfVectorizer (min_df=4, max_df=0.1, max_features=10000, stop_words=russian_stopwords, ngram_range=(1, 3))
X_for_RFC = vectorizerRFC.fit_transform(train.comment)
X_for_RFC_test = vectorizerRFC.transform(test.comment)

vectorizerT = TfidfVectorizer (min_df=4, max_df=0.1, max_features=10000, stop_words=russian_stopwords, ngram_range=(1, 3))
X_for_T = vectorizerT.fit_transform(train.comment)
X_for_T_test = vectorizerT.transform(test.comment)

In [92]:
yRFC = train.toxic.values
yRFC_test = test.toxic.values
yT = train.toxic.values
yT_test = test.toxic.values

In [93]:
clfRFC = RFC(min_samples_leaf = 2, class_weight = 'balanced')
clfRFC.fit(X_for_RFC, yRFC)
predsRFC = clfRFC.predict(X_for_RFC_test)

clfT = T(min_samples_leaf = 2, class_weight = 'balanced')
clfT.fit(X_for_T, yT)
predsT = clfT.predict(X_for_T_test)

In [94]:
counter_tRFC = Counter({tok: feature
        for tok, feature
        in zip(
            vectorizerRFC.vocabulary_,
            clfRFC.feature_importances_
        )}
)
counter_tRFC.most_common(5)

[('коммунизм', np.float64(0.013566331604065696)),
 ('свалить', np.float64(0.010518633594516874)),
 ('учли', np.float64(0.010145038801353685)),
 ('выдержать', np.float64(0.00942615388933382)),
 ('10 11', np.float64(0.0076900969257069065))]

In [95]:
counter_tT = Counter({tok: feature
        for tok, feature
        in zip(
            vectorizerT.vocabulary_,
            clfT.feature_importances_
        )}
)
counter_tT.most_common(5)

[('коммунизм', np.float64(0.018218810777401614)),
 ('учли', np.float64(0.018174531334944483)),
 ('свалить', np.float64(0.01387441049045727)),
 ('вероятность', np.float64(0.013387477585772065)),
 ('похоже это', np.float64(0.011182689848971307))]